Copyright © 2021-2022 HQS Quantum Simulations GmbH. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except
in compliance with the License. You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.

# Quantum Teleportation with roqoqo & the use of conditional measurements

This notebook is designed to demonstrate the use of conditional measurements, by following through an example of quantum state teleportation.

In quantum teleportation there are two end users: The first user, Alice, wishes to send a particular quantum state to the second user, Bob. The protocol requires a total of three qubits, and the transmission of two classical bits. 

The sender Alice controls qubits 0 and 1, and the reciever Bob controls qubit 2. 

In [2]:
:dep roqoqo = "1.0.0-beta.2"
:dep roqoqo-quest = "0.7.2"
:dep num-complex = "0.4"
:dep qoqo_calculator = "1.0.0"

In [3]:
extern crate num_complex;
extern crate roqoqo;
extern crate roqoqo_quest;

use roqoqo::{Circuit, operations as ops,prelude::*};
use roqoqo_quest::Backend;
use std::collections::HashMap;
use qoqo_calculator::CalculatorFloat;
use core::f64::consts::PI as Pi;

## State preparation

The first step is to prepare the quantum state which Alice will send to Bob. As an example, the most general single qubit quantum state is given by:
\begin{equation}
|\psi \rangle = cos(\frac{\theta}{2}) |0 \rangle + e^{i \phi} sin(\frac{\theta}{2}) |1 \rangle.
\end{equation}
This state can be prepared by a sequence of two single qubit rotations. In the code block below we first define a function that takes the angles $\theta$ and $\phi$ as input and prepares qubit 0 of a quantum register in the state $| \psi \rangle$.

Next we use an instance of the function with the angles $\theta=\frac{\pi}{2}$ and $\phi=0$ to create a circuit which prepares the state: 
\begin{equation}
|\psi \rangle = \frac{1}{\sqrt{2}} \big ( |0 \rangle + |1 \rangle \big ) = | + \rangle.
\end{equation}

In [4]:
fn prep_psi (angle_thet: CalculatorFloat, angle_phi: CalculatorFloat) -> Circuit
  {
    let mut circuit = Circuit::new();
    circuit += ops::RotateY::new(0, angle_thet);
    circuit += ops::RotateZ::new(0, angle_phi);
    return circuit
}

let init_circuit = prep_psi(CalculatorFloat::Float(Pi/2f64), CalculatorFloat::Float(0f64));

## Preparing an entangled resource state

Quantum teleportation requires that the end users initially share an entangled resource state, 
\begin{equation}
|\Phi_{+} \rangle = \frac{1}{\sqrt(2)} \big ( |00 \rangle + |11 \rangle \big ) .
\end{equation}

The following circuit prepares the state $|\Phi_{+} \rangle$ between qubit 1, held by Alice, and qubit 2, held by Bob.

In [5]:
let mut entangling_circ = Circuit::new();
entangling_circ += ops::Hadamard::new(1);
entangling_circ += ops::CNOT::new(1, 2);

## Encoding the state to be sent in the entangled resource state

The next step of the procedure is to encode the state of qubit 0, $\psi$, into the entangled resource state. This is accomplished by way of the circuit defined below, which is similar to that used to prepare the entangled resource. 

In [6]:
let mut encoding_circ = Circuit::new();
encoding_circ += ops::CNOT::new(0, 1);
encoding_circ += ops::Hadamard::new(0);

## State transfer part 1: Measurement

At this stage in the process both of Alice's qubits, 0 and 1, are measured. The measurement consumes the entangled resource and leaves the state of qubit 2,Bob's qubit, in a state that depends on the two measurement outcomes. 

Let us call the classical bit which results from measuring qubit 0 'M1' and the bit resulting from measuring qubit 1 'M2'. The circuit below defines the classical register named 'M1M2', performs the measurement of qubits 0 and 1, and stores the results in the register 'M1M2'. 

In [7]:
let mut meas_circ = Circuit::new();
meas_circ += ops::DefinitionBit::new("M1M2".to_string(), 2, true);
meas_circ += ops::MeasureQubit::new(0, "M1M2".to_string(), 0);
meas_circ += ops::MeasureQubit::new(1, "M1M2".to_string(), 1);

## Defining the circuit for a conditional operation

Conditional operations in roqoqo have three inputs: the name of a classical register containing boolean values, the index of the register containing the value to be used to condition the operation, and the operation or sequence of operations to be performed if the boolean condition value is True. 

To prepare the third input, it is necessary to create circuit snippets corresponding to the operations to be completed if the condition is True. 

In the case of quantum teleportation, we need two conditional operations. The first is a Pauli Z acting on Bob's qubit, conditioned on the measurement result M1. The second is a Pauli X acting on Bob's qubit, conditioned on the measurement result M2. 

Hence we prepare circuit snippets correspponding to a Pauli Z and a Pauli X operation.

In [8]:
let mut conditional_z = Circuit::new();
conditional_z += ops::PauliZ::new(2);

let mut conditional_x = Circuit::new();
conditional_x += ops::PauliX::new(2);

## State transfer part 2: conditional operations

The final stage of the teleportation protocol is to perform corrections to the state of Bob's qubit 2, according to the measurement outcomes 'M1' and 'M2'.

The below circuit makes use of the circuit snippets defined above to perform the conditional corrections to the state of qubit 2. 2. 

In [9]:
let mut conditional_circ = Circuit::new();
conditional_circ += ops::PragmaConditional::new("M1M2".to_string(), 1, conditional_x);
conditional_circ += ops::PragmaConditional::new("M1M2".to_string(), 0, conditional_z);

## Putting it all together

Combining each of the circuits we have defined yeilds the full teleportation protocol. We can verify that the protocol is successful by reading out the final state vector and comparing it to the state which was to be sent, $|\psi \rangle$.

In [10]:
let mut verification = Circuit::new();
verification += ops::DefinitionComplex::new("psi".to_string(), 8, true);
verification += ops::PragmaGetStateVector::new("psi".to_string(), Some(Circuit::new()));

let teleportation_circuit = init_circuit + entangling_circ + encoding_circ + meas_circ + conditional_circ + verification;

let backend = Backend::new(3);
let result_of_run = backend.run_circuit(&teleportation_circuit);
let (result_bit_registers, _result_float_registers, result_complex_registers) 
= result_of_run.unwrap();

println!("Result bit registers :{:?}",result_bit_registers["M1M2"]);
println!("Result complex registers :{:?}",result_complex_registers["psi"]);

Result bit registers :[[true, true]]
Result complex registers :[[Complex { re: 0.0, im: 0.0 }, Complex { re: 0.0, im: 0.0 }, Complex { re: 0.0, im: 0.0 }, Complex { re: 0.7071067811865472, im: 0.0 }, Complex { re: -0.0, im: 0.0 }, Complex { re: -0.0, im: 0.0 }, Complex { re: -0.0, im: 0.0 }, Complex { re: 0.7071067811865471, im: -0.0 }]]
